# Week 6 Homework

***Due (pushed to your GitHub branch) on 10/24 by 11:59 pm***

## STAR two-pass alignment

Build the STAR SA:

In [1]:
%mkdir alignment/star_for_homework/
%mkdir ../2_genome_exploration/genome_for_homework/star_for_homework

!STAR \
--runThreadN 32 \
--runMode genomeGenerate \
--genomeDir ../2_genome_exploration/genome_for_homework/star_for_homework \
--genomeFastaFiles ../2_genome_exploration/genome_for_homework/genome_for_homework.fa \
--sjdbGTFfile ../2_genome_exploration/genome_for_homework/annotations_for_homework.gtf \
--sjdbOverhang 50 \
--genomeSAindexNbases 13

	STAR --runThreadN 32 --runMode genomeGenerate --genomeDir ../2_genome_exploration/genome_for_homework/star_for_homework --genomeFastaFiles ../2_genome_exploration/genome_for_homework/genome_for_homework.fa --sjdbGTFfile ../2_genome_exploration/genome_for_homework/annotations_for_homework.gtf --sjdbOverhang 50 --genomeSAindexNbases 13
	STAR version: 2.7.10b   compiled: 2022-11-01T09:53:26-04:00 :/home/dobin/data/STAR/STARcode/STAR.master/source
Oct 21 00:28:11 ..... started STAR run
Oct 21 00:28:11 ... starting to generate Genome files
Oct 21 00:29:37 ..... processing annotations GTF
Oct 21 00:30:25 ... starting to sort Suffix Array. This may take a long time...
Oct 21 00:30:38 ... sorting Suffix Array chunks and saving them to disk...
Oct 21 00:36:56 ... loading chunks from disk, packing SA...
Oct 21 00:41:59 ... finished generating suffix array
Oct 21 00:41:59 ... generating Suffix Array index
Oct 21 00:44:15 ... completed Suffix Array index
Oct 21 00:44:16 ..... inserting junctions 

Use STAR and ***all*** of your FASTQs for a first-pass alignment. Be sure to update the `sjdbOverhang` parameter to a value that is appropriate for your dataset. Rename the log file so that you can compare the logs from the first and second alignments:

In [4]:
!STAR \
--runThreadN 32 \
--runMode alignReads \
--genomeDir ../2_genome_exploration/genome_for_1/star_for_homework \
--readFilesIn ../5_fastq/mouse_hippocampus_fastq/SRR2916027.fastq.gz ../5_fastq/mouse_hippocampus_fastq/SRR2916049.fastq.gz \
--readFilesCommand zcat \
--outSAMtype BAM Unsorted \
--outSAMunmapped Within \
--outFileNamePrefix alignment/star_for_homework/first_pass_ \
--sjdbOverhang 50

	STAR --runThreadN 32 --runMode alignReads --genomeDir ../2_genome_exploration/genome_for_1/star_for_homework --readFilesIn ../5_fastq/mouse_hippocampus_fastq/SRR2916027.fastq.gz ../5_fastq/mouse_hippocampus_fastq/SRR2916049.fastq.gz --readFilesCommand zcat --outSAMtype BAM Unsorted --outSAMunmapped Within --outFileNamePrefix alignment/star_for_homework/first_pass_ --sjdbOverhang 50
	STAR version: 2.7.10b   compiled: 2022-11-01T09:53:26-04:00 :/home/dobin/data/STAR/STARcode/STAR.master/source
Oct 21 01:01:38 ..... started STAR run
Oct 21 01:01:38 ..... loading genome

EXITING because of FATAL ERROR: could not open genome file ../2_genome_exploration/genome_for_1/star_for_homework//genomeParameters.txt
SOLUTION: check that the path to genome files, specified in --genomeDir is correct and the files are present, and have user read permsissions

Oct 21 01:01:38 ...... FATAL ERROR, exiting


Use STAR, all of your FASTQs, and `SJ.out.tab` for the second-pass alignment. You will likely need to make your own `manifest.tsv`:

In [5]:
!STAR \
--runThreadN 32 \
--runMode alignReads \
--genomeDir ../2_genome_exploration/genome_for_homework/star_for_homework \
--readFilesManifest manifest.tsv \
--readFilesCommand zcat \
--outSAMtype BAM Unsorted \
--outSAMunmapped Within \
--outFileNamePrefix alignment/star_for_homework/

	STAR --runThreadN 32 --runMode alignReads --genomeDir ../2_genome_exploration/genome_for_homework/star_for_homework --readFilesManifest manifest.tsv --readFilesCommand zcat --outSAMtype BAM Unsorted --outSAMunmapped Within --outFileNamePrefix alignment/star_for_homework/
	STAR version: 2.7.10b   compiled: 2022-11-01T09:53:26-04:00 :/home/dobin/data/STAR/STARcode/STAR.master/source
Oct 21 15:12:06 ..... started STAR run
Oct 21 15:12:06 ..... loading genome
Oct 21 15:13:51 ..... started mapping


Update this block to briefly describe the differences between the logs for the first and second alignments.

## HISAT alignment

(***Note***: you may need Dr. Wheeler to build the HISAT index of your genome. Ask him for assistance when you get to this part of the assignment.)

Use a Bash `while` loop to align all of your FASTQs and save the output files using the same names as the input FASTQ:

In [ ]:
%mkdir alignment/hisat_for_homework/

!while read -r line; do \
    fq1=$(echo $line | awk '{print $1}'); \
    fq2=$(echo $line | awk '{print $2}'); \
    bn=$(basename "$fq1" .fastq.gz) \
    bn=$(echo "$bn" | sed 's/_1$//') \
    rg=$(echo $line | awk '{print $3}' | sed 's/ID://' ); \
    echo "Aligning $fq1 and $fq2 with $rg"; \
    hisat2 /data/users/zhouz6436/BIOL343/6_alignment/hisat -p 16 -1 $fq1 -2 $fq2 --rg-id $rg --rg SM:$rg --summary-file alignment/hisat/$bn.log --new-summary > alignment/hisat/$bn.sam; \
    done < manifest.tsv

## Snakemake Pipeline

In contrast to last week (FASTQ files), there aren't as many decisions that need to be made during alignment, so using a notebook to control and report code isn't as necessary. As you saw in in [6_alignment](6_alignment.ipynb), file and directory management can get complicated during alignment. This is a case where Snakemake scripting shines.

1. Copy your `Snakefile` and `config.yaml` from `5_fastq` to `6_alignment`. 
2. In the `Snakefile`, add the following rules:  
    a. `fetch_genome` - using a variation from the code from `2_genome_exploration`, download your reference genome and annotations.  
    b. `star_sa` - use `STAR` to produce the suffix array of the reference genome.  
    c. `star_align_first` - use `STAR` to perform first-pass alignment with all FASTQ files.  
    d. `star_align_second` - use `STAR` to perform second-pass alignment with all FASTQ files.  
    e. `hisat_align` - use `HISAT` to align all FASTQ files.  
    f. `report` - use `multiqc` to aggregate all QC data into a single report.

This `Snakefile` should not use any `wrappers` because we already have the relevant software installed in the `biol343` conda environment. The pipeline should run to completion when any instructor or classmate runs `snakemake --use-conda`.